In [17]:
%pip install --upgrade --quiet boto3 botocore langchain cohere python-dotenv

In [64]:
import boto3
import os
import json

from langchain.embeddings.cohere import CohereEmbeddings

from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, PyPDFDirectoryLoader

from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [66]:
loader = PyPDFDirectoryLoader("./kr_doc/")

In [52]:
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = 1000,
    chunk_overlap  = 100,
)
docs = text_splitter.split_documents(documents)

In [67]:
embeddings = CohereEmbeddings(model = "multilingual-22-12")
vectorstore_faiss = FAISS.from_documents(
    docs,
    embeddings
)

In [74]:
#query = """How to clean the product?"""
query = """제품을 청소하는 방법?"""

In [75]:
docs = vectorstore_faiss.similarity_search(query)

In [76]:
print(docs)

[Document(page_content='제품을 청소할 때 주의하세요.\n •제품을 청소할 때는 연필용 지우개나 부드러운 천으로 가볍게 닦으세요.\n •제품의 충전 단자를 청소할 때는 면봉이나 부드러운 천으로 닦으세요.\n •제품을 청소하기 위해 독한 화학 물질이나 강한 세제 등을 사용하지 마세요. 제품의 외관이 \n변색되거나 부식될 수 있으며, 화재 또는 감전의 위험이 있습니다.\n •먼지, 땀, 잉크, 기름, 화학 제품(화장품, 항균 스프레이, 손 세정제, 세제, 살충제)이 닿지 않도록 \n주의하세요. 제품의 외관이나 내부 부품이 손상되어 성능이 저하될 수 있습니다. 해당 물질이 \n묻은 경우엔 보풀이 없는 부드러운 천으로 닦으세요.', metadata={'source': 'kr_doc/kr_safety_information.pdf', 'page': 8}), Document(page_content='•자동차에 설치된 휴대용 기기나 관련 장치들이 단단히 고정되었는지 확인하세요.\n •에어백이 장착된 차량에서 사용할 경우 제품을 에어백 주변에 설치하거나 보관하지 마세요. \n에어백 작동 시 제품이 파손되거나 제품에 부딪혀 다칠 수 있습니다.', metadata={'source': 'kr_doc/kr_safety_information.pdf', 'page': 7}), Document(page_content='•제품을 너무 낮거나 너무 높은 온도에서 보관하지 마세요.\n •제품을 너무 높은 온도에서 보관하면 제품이 고장 나거나 배터리 수명이 단축될 수 있습니다.\n •배터리의 음극 단자와 양극 단자를 직접 연결하지 말고, 배터리에 금속 물체가 닿지 않게 \n하세요. 배터리가 고장 날 수 있습니다.', metadata={'source': 'kr_doc/kr_safety_information.pdf', 'page': 1}), Document(page_content='기기인지 확인 후 사용하세요.\n심한 매연이나 증기를 피하세요.\n제품 외관이 훼손되거나 고장 날 수

In [57]:
# Create our own prompt template

prompt_template = """Text: {context}

Question: {question}

Answer the question based on the text provided. If the text doesn't contain the answer, reply that the answer is not available."""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
modelId = 'cohere.command-text-v14'
accept = 'application/json'
contentType = 'application/json'

In [ ]:
bedrock_runtime = boto3.client(
    service_name = "bedrock-runtime",
    region_name = "us-east-1"
)

In [58]:
def get_inference_parameters(model): #return a default set of parameters based on the model's provider
    bedrock_model_provider = model.split('.')[0] #grab the model provider from the first part of the model id
    
    if (bedrock_model_provider == 'anthropic'): #Anthropic model
        return { #anthropic
            "max_tokens_to_sample": 512,
            "temperature": 0, 
            "top_k": 250, 
            "top_p": 1, 
            "stop_sequences": ["\n\nHuman:"] 
           }
    
    elif (bedrock_model_provider == 'ai21'): #AI21
        return { #AI21
            "maxTokens": 512, 
            "temperature": 0, 
            "topP": 0.5, 
            "stopSequences": [], 
            "countPenalty": {"scale": 0 }, 
            "presencePenalty": {"scale": 0 }, 
            "frequencyPenalty": {"scale": 0 } 
           }
    
    elif (bedrock_model_provider == 'cohere'): #COHERE
        return {
            "max_tokens": 512,
            "temperature": 0,
            "p": 0.01,
            "k": 0,
            "stop_sequences": [],
            "return_likelihoods": "NONE"
        }
    
    else: #Amazon
        #For the LangChain Bedrock implementation, these parameters will be added to the 
        #textGenerationConfig item that LangChain creates for us
        return { 
            "maxTokenCount": 512, 
            "stopSequences": [], 
            "temperature": 0, 
            "topP": 0.9 
        }

In [59]:
from langchain.llms.bedrock import Bedrock

model_kwargs = get_inference_parameters("cohere")
llm = Bedrock(
        model_id=modelId, #use the requested model
        model_kwargs = model_kwargs
    )

In [60]:
chain_type_kwargs = {"prompt": PROMPT}

qa = RetrievalQA.from_chain_type(llm=llm, 
                                 chain_type="stuff", 
                                 retriever=vectorstore_faiss.as_retriever(), 
                                 chain_type_kwargs=chain_type_kwargs, 
                                 return_source_documents=True)

In [61]:
query = """How to clean the product"""

In [62]:
answer = qa({"query": query})
result = answer["result"].replace("\n","").replace("Answer:","")
print(f"Question: {query}")
print(f"Answer: {result}")

Question: How to clean the product
Answer:  The text provides instructions for cleaning the product. Here is a summary of the instructions:- Clean the product gently with a soft cloth or a brush.- Clean the battery terminals with a soft cloth or a brush.- Do not use strong chemicals or abrasive cleaners when cleaning the product. - Be careful not to damage the external appearance or internal components of the product.- If the product comes into contact with dust, sweat, ink, oil, or chemical products, clean it gently with a soft dry cloth.The text also provides some additional guidelines for handling the product:- Check that any installed portable devices or related accessories have been securely fixed.- Do not place the product in an environment with extreme temperatures.- Do not expose the product to high temperatures, as this may cause it to malfunction or reduce its battery life.- Avoid direct contact between the positive and negative terminals of the battery.- Do not use the produ